In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/rgb_hand

/content/drive/MyDrive/Colab Notebooks/rgb_hand


In [ ]:
import pandas as pd
import cv2
import numpy as np
from keras import Sequential
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load the CSV file into a pandas DataFrame
ds = pd.read_csv("dorsal_right_30.csv")

In [ ]:
# Create a dictionary to store the count of images for each ID
id_counts = {}

# Iterate through the DataFrame to count the number of images for each ID
for idx, row in ds.iterrows():
    id_num = row["ID"]
    if id_num in id_counts:
        id_counts[id_num] += 1
    else:
        id_counts[id_num] = 1

# Create a list to store the indices of the test data
test_indices = []

# Iterate through the DataFrame to find the last 2 images for each ID
for idx, row in ds.iterrows():
    id_num = row["ID"]
    if id_counts[id_num] > 2:
        id_counts[id_num] -= 1
    else:
        test_indices.append(idx)

# Split the DataFrame into training and testing datasets
train_ds = ds.drop(test_indices)
test_ds = ds.iloc[test_indices]

In [ ]:
# Pre-process the images and resize them
def preprocess_image2(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    return img

In [ ]:
from cv2.ximgproc import guidedFilter

# Pre-process the images to low and high frequency images and resize them
def preprocess_image(img_path):

    img = cv2.imread(img_path)

    # Normalize the image
    norm = np.array(img, dtype=np.float32) / 255.0
    
    # Apply the guided filter
    low_img = guidedFilter(norm, norm, 10, 0.05)

    #Convert the image to BGR
    bgr = cv2.cvtColor(low_img, cv2.COLOR_RGB2BGR)

    high_img = np.uint8(cv2.normalize(np.divide(cv2.cvtColor(norm, cv2.COLOR_RGB2BGR), bgr), None, 0, 255, cv2.NORM_MINMAX))

    # Resize the image
    low_img = cv2.resize(low_img, (224, 224))
    high_img = cv2.resize(high_img, (224,224))


    return low_img, high_img

In [ ]:
# Prepare the train and test data
train_low = []
train_high = []
for i, row in train_ds.iterrows():
    img_path = "/content/drive/MyDrive/Colab Notebooks/rgb_hand/hands/" + row["ImageName"]
    low_img, high_img = preprocess_image(img_path)
    train_low.append(low_img)
    train_high.append(high_img)

train_low = np.array(train_low)
train_high = np.array(train_high)
train_labels = train_ds["ID"].values

test_low = []
test_high = []
for i, row in test_ds.iterrows():
    img_path = "/content/drive/MyDrive/Colab Notebooks/rgb_hand/hands/" + row["ImageName"]
    low_img, high_img = preprocess_image(img_path)
    test_low.append(low_img)
    test_high.append(high_img)

test_low = np.array(test_low)
test_high = np.array(test_high)
test_labels = test_ds["ID"].values

In [ ]:
import keras
from keras.applications import VGG16
from keras.layers import Input, Concatenate, Dense, Flatten
from keras.models import Model
from tensorflow.keras.optimizers import Adam

# Split the train data into training and validation sets
train_low, val_low, train_high, val_high, train_labels, val_labels = train_test_split(train_low, train_high, train_labels, test_size=0.15, random_state=42)


# define the shape of the input images
input_shape = (224, 224, 3)

# create the two streams for image sets 1 and 2
input1 = Input(shape=input_shape)
input2 = Input(shape=input_shape)

# pre-trained VGG16 network for both streams
vgg16_model1 = VGG16(weights='imagenet', include_top=False, input_tensor=input1)
vgg16_model2 = VGG16(weights='imagenet', include_top=False, input_tensor=input2)

# add prefix to the layer names in the second stream
for layer in vgg16_model2.layers:
    layer._name = 'stream2_' + layer.name

# merge the two streams by concatenating their output
merged = Concatenate()([vgg16_model1.output, vgg16_model2.output])

# add additional dense layers to handle hand identification
x = Flatten()(merged)
x = Dense(1096, activation='relu')(x)
x = Dense(1096, activation='relu')(x)
x = Dense(30, activation='softmax')(x)

# create the final model
model = Model(inputs=[input1, input2], outputs=x)

# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])


58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
# Train the model
history = model.fit([train_low, train_high], train_labels, epochs=25, batch_size=12, validation_data=([val_low, val_high], val_labels))

Epoch 1/25
12/12 [==============================] - 32s 757ms/step - loss: 11.6441 - accuracy: 0.0507 - val_loss: 3.2862 - val_accuracy: 0.0800
Epoch 2/25
12/12 [==============================] - 4s 336ms/step - loss: 3.4097 - accuracy: 0.0435 - val_loss: 3.4673 - val_accuracy: 0.0400
Epoch 3/25
12/12 [==============================] - 4s 330ms/step - loss: 3.4538 - accuracy: 0.0652 - val_loss: 3.3284 - val_accuracy: 0.0800
Epoch 4/25
12/12 [==============================] - 4s 329ms/step - loss: 3.4737 - accuracy: 0.0725 - val_loss: 3.3214 - val_accuracy: 0.0400
Epoch 5/25
12/12 [==============================] - 4s 329ms/step - loss: 3.3560 - accuracy: 0.0942 - val_loss: 3.2557 - val_accuracy: 0.1600
Epoch 6/25
12/12 [==============================] - 4s 333ms/step - loss: 3.2860 - accuracy: 0.0942 - val_loss: 3.3215 - val_accuracy: 0.0800
Epoch 7/25
12/12 [==============================] - 4s 337ms/step - loss: 3.2598 - accuracy: 0.1014 - val_loss: 3.3681 - val_accuracy: 0.1200
Epoc

In [ ]:
# Evaluate the model on two stream network
test_loss, test_acc = model.evaluate([test_low, test_high], test_labels)
print('Test Accuracy: ', test_acc)
print('Test loss:', test_loss)

2/2 [==============================] - 10s 5s/step - loss: 3.4329 - accuracy: 0.0333
Test Accuracy:  0.03333333507180214
Test loss: 3.432859182357788
